In [113]:
import sys
import numpy as np
import gmsh
from collections import defaultdict

# Initialize the Gmsh API
gmsh.initialize()

# Create a new model
gmsh.model.add("Unit Square")

# Define the vertices of the square (points)
p1 = gmsh.model.geo.addPoint(0, 0, 0)  # Point (0, 0)
p2 = gmsh.model.geo.addPoint(1, 0, 0)  # Point (1, 0)
p3 = gmsh.model.geo.addPoint(1, 1, 0)  # Point (1, 1)
p4 = gmsh.model.geo.addPoint(0, 1, 0)  # Point (0, 1)

# Create the lines (edges) of the square
l1 = gmsh.model.geo.addLine(p1, p2)  # Line between (0, 0) and (1, 0)
l2 = gmsh.model.geo.addLine(p2, p3)  # Line between (1, 0) and (1, 1)
l3 = gmsh.model.geo.addLine(p3, p4)  # Line between (1, 1) and (0, 1)
l4 = gmsh.model.geo.addLine(p4, p1)  # Line between (0, 1) and (0, 0)

# Create a curve loop to define the boundary of the square
loop = gmsh.model.geo.addCurveLoop([l1, l2, l3, l4])

# Create a plane surface for the square
surface = gmsh.model.geo.addPlaneSurface([loop])

# Synchronize the CAD kernel
gmsh.model.geo.synchronize()

# Define a mesh size field (optional)
# You can define a uniform or non-uniform mesh size; here we use uniform
gmsh.model.mesh.generate(2)  # Generate a 2D mesh

# Save the mesh to a file (e.g., in .msh format)
gmsh.write("unit_square.msh")

if '-nopopup' not in sys.argv:
    gmsh.fltk.run()

# Finalize and close the Gmsh API
gmsh.finalize()

In [119]:
# Play around with mesh entities
import sys
import numpy as np
import networkx as nx
import gmsh
from collections import defaultdict

# Initialize Gmsh
gmsh.initialize()

# Open the mesh file
gmsh.open("unit_square.msh")

visualize = False
if visualize and '-nopopup' not in sys.argv:
    gmsh.fltk.run()

entities = gmsh.model.getEntities()

for e in entities:
    dim = e[0]
    tag = e[1]

    # Get the mesh nodes for the entity (dim, tag)
    nodeTags, nodeCoords, nodeParams = gmsh.model.mesh.getNodes(dim, tag)

    # Get the mesh elements for the entity (dim, tag)
    elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(dim, tag)

    # * Type and name of the entity:
    type = gmsh.model.getType(dim, tag)
    name = gmsh.model.getEntityName(dim, tag)
    if len(name): name += ' '
    print("Entity " + name + str(e) + " of type " + type)

    # * Number of mesh nodes and elements:
    numElem = sum(len(i) for i in elemTags)
    print(" - Mesh has " + str(len(nodeTags)) + " nodes and " + str(numElem) +
          " elements")

    # * Upward and downward adjacencies:
    up, down = gmsh.model.getAdjacencies(dim, tag)
    if len(up):
        print(" - Upward adjacencies: " + str(up))
    if len(down):
        print(" - Downward adjacencies: " + str(down))

    # * Does the entity belong to physical groups?
    physicalTags = gmsh.model.getPhysicalGroupsForEntity(dim, tag)
    if len(physicalTags):
        s = ''
        for p in physicalTags:
            n = gmsh.model.getPhysicalName(dim, p)
            if n: n += ' '
            s += n + '(' + str(dim) + ', ' + str(p) + ') '
        print(" - Physical groups: " + s)

    # * Is the entity a partition entity? If so, what is its parent entity?
    partitions = gmsh.model.getPartitions(dim, tag)
    if len(partitions):
        print(" - Partition tags: " + str(partitions) + " - parent entity " +
              str(gmsh.model.getParent(dim, tag)))

    # * List all types of elements making up the mesh of the entity:
    for t in elemTypes:
        name, dim, order, numv, parv, _ = gmsh.model.mesh.getElementProperties(
            t)
        print(" - Element type: " + name + ", order " + str(order) + " (" +
              str(numv) + " nodes in param coord: " + str(parv) + ")")

# Finalize Gmsh
gmsh.finalize()

Entity (0, 1) of type Point
 - Mesh has 1 nodes and 1 elements
 - Upward adjacencies: [1 4]
 - Element type: Point, order 0 (1 nodes in param coord: [0.])
Entity (0, 2) of type Point
 - Mesh has 1 nodes and 1 elements
 - Upward adjacencies: [1 2]
 - Element type: Point, order 0 (1 nodes in param coord: [0.])
Entity (0, 3) of type Point
 - Mesh has 1 nodes and 1 elements
 - Upward adjacencies: [2 3]
 - Element type: Point, order 0 (1 nodes in param coord: [0.])
Entity (0, 4) of type Point
 - Mesh has 1 nodes and 1 elements
 - Upward adjacencies: [3 4]
 - Element type: Point, order 0 (1 nodes in param coord: [0.])
Entity (1, 1) of type Discrete curve
 - Mesh has 7 nodes and 8 elements
 - Upward adjacencies: [1]
 - Downward adjacencies: [1 2]
 - Element type: Line 2, order 1 (2 nodes in param coord: [-1.  1.])
Entity (1, 2) of type Discrete curve
 - Mesh has 7 nodes and 8 elements
 - Upward adjacencies: [1]
 - Downward adjacencies: [2 3]
 - Element type: Line 2, order 1 (2 nodes in param 

In [169]:
import sys
import numpy as np
import networkx as nx
import gmsh
from collections import defaultdict

# Initialize Gmsh
gmsh.initialize()

# Open the mesh file
gmsh.open("unit_square.msh")

# Extract node data
node_tags, node_coords, _ = gmsh.model.mesh.getNodes()

# node_tags: list of node IDs (integers)
# node_coords: list of node coordinates (flattened list: x1, y1, z1, x2, y2, z2, ...)
# The z-coordinates will be 0 for a 2D mesh

# Convert the flattened list of node coordinates into a dictionary
# This maps node ID to its (x, y, z) coordinates
node_coords_dict = {node_tags[i]: (node_coords[3*i], node_coords[3*i+1], node_coords[3*i+2]) 
                    for i in range(len(node_tags))}

# Function to get the coordinates of the nodes of a triangle
def get_tri_coords(triangle):
    coords = np.array([node_coords_dict[node_id] for node_id in triangle])
    # Since these are 2D triangles, drop the z-coordinates
    coords = np.array([coord[0:2] for coord in coords])
    return coords

# Function to get the direction vectors of each edge in a triangle
def get_tri_edges(triangle):
    coords = get_tri_coords(triangle)
    edge1 = coords[1] - coords[0]
    edge2 = coords[2] - coords[1]
    edge3 = coords[0] - coords[2]
    return np.array([edge1, edge2, edge3])

# Compute outward pointing normal for each edge in a triangle
def get_normals(triangle):
    coords = get_tri_coords(triangle)
    edges  = get_tri_edges(triangle)

    # Compute the midpoints of each edge
    midpts = coords + 0.5*edges

    # Compute the centroid of the triangle
    centroid = (1.0/3.0)*np.sum(coords, axis=0)

    # Compute direction vectors from centroid to midpoint of each edge
    midpts_dirs = midpts - centroid

    # Compute normals to each edge, and check dot product of normals
    # with centroid
    normals = edges[:,[1, 0]]
    normals[:, 0] *= -1.0
    # If the dot product is negative, negate the normal
    owp_normals = np.array([normals[i, :] if np.dot(normals[i, :], midpts_dirs[i, :]) >= 0 \
                            else -normals[i, :] for i in range(3)])
    # Normalize the o.w.p. normals
    owp_normals = owp_normals / np.linalg.norm(owp_normals, axis=1).reshape(-1, 1)
    return owp_normals

# Extract element data (triangular elements)
element_types, element_tags, element_node_tags = gmsh.model.mesh.getElements()

# for e_type, e_ids in zip(element_types, element_tags):
#     print(f"Element Type: {e_type}, Element IDs: {e_ids}")

# element_types: list of element types (e.g., 2 is for lines, 3 is for triangles)
# element_tags: list of element IDs (integers)
# element_node_tags: connectivity of nodes for each element (flattened list)

# Extract triangle element tags
triangle_tags  = element_tags[1] # Assuming the second sub-array of element tags corresponds to triangles 
num_triangles  = len(triangle_tags)

# We are interested in triangles, which have type 2 (for triangles in 2D)
# element_node_tags is a list of node indices defining the triangular elements
triangle_nodes = element_node_tags[1]  # Assuming the second element type corresponds to triangles

# Convert the flattened node list into tuples of three nodes (triangles)
triangles = [(triangle_nodes[i], triangle_nodes[i+1], triangle_nodes[i+2]) for i in range(0, len(triangle_nodes), 3)]

# Create a dict that maps each element tag to its node tags,
# and vice versa
triangles_to_nodes = dict()
nodes_to_triangles = dict()
for j in np.arange(num_triangles):
    triangles_to_nodes[element_tags[1][j]] = [triangles[j]]
    nodes_to_triangles[triangles[j]]       = element_tags[1][j]

# Extract the boundary triangles and boundary edges

# Create a dictionary to store the edges and count their occurrences
# edge_count        = defaultdict(int)

# Create a dictionary to store edges and the corresponding triangles that
# contain them, and vice versa
edge_to_triangles  = defaultdict(list)
triangles_to_edges = defaultdict(list)

# Loop over each triangle, and extract its edges
for i, triangle in enumerate(triangles):
    # A triangle has 3 edges: (node1, node2), (node2, node3), (node3, node1)
    edges = [
        tuple(sorted([triangle[0], triangle[1]])), # Edge 1-2
        tuple(sorted([triangle[1], triangle[2]])), # Edge 2-3
        tuple(sorted([triangle[2], triangle[0]]))  # Edge 3-1
    ]

    for edge in edges:
        # For each triangle, store the triangle index
        # edge_to_triangles[edge].append(i)
        edge_to_triangles[edge].append(element_tags[1][i]) # Use the element tags instead of natural count
        triangles_to_edges[element_tags[1][i]].append(edge)
        # Increment the count for each edge
        # edge_count[edge] += 1

# Now, find the boundary edges (those that appear only once)
# boundary_edges = [edge for edge, count in edge_count.items() if count == 1]
boundary_edges = [edge for edge, triangles in edge_to_triangles.items() if len(triangles) == 1]

# Now, retrieve the triangles that contain these boundary edges
boundary_triangles = set() # Using a set to avoid duplicates

for edge in boundary_edges:
    # Get the triangle containing this edge
    triangle_indices = edge_to_triangles[edge]

    # Since it's a boundary edge, it must belong to exactly one triangle
    boundary_triangles.add(triangle_indices[0])

# For each triangle, determine its neighbors
triangle_nbrs = defaultdict(set) # To avoid duplicate neighbors
for edge, shared_triangles in edge_to_triangles.items():
    if len(shared_triangles) > 1:
        t1 = shared_triangles[0]
        t2 = shared_triangles[1]
        triangle_nbrs[t1].add(t2)
        triangle_nbrs[t2].add(t1)

# Create discrete directions 
num_angles = 30
angles     = np.linspace(0, 2*np.pi, num_angles)

# Loop over directions
for i, angle in enumerate(angles):
    dir = np.array([np.cos(angle), np.sin(angle)])
    # Track the order in which we can solve over the triangles
    dir_order = dict()
    # Loop over triangles
    for j, triangle in enumerate(triangles):
        tri_elem = element_tags[1][j]
        # If triangle is on boundary, we can immediately solve over it
        if element_tags[1][j] in boundary_triangles:
            dir_order[tri_elem] = 0
        else:
            coords      = get_tri_coords(triangle)
            edges       = get_tri_edges(triangle)
            owp_normals = get_normals(triangle)

        # for k, edge in enumerate(edges):
        #     # We impose BCs on boundary edges, so skip them
        #     is_b_edge = np.any(np.all(boundary_edges == edge, axis=1))
        #     if not is_b_edge:
        #         # Determine if edge is either an inflow edge
        #         # or outflow edge
        #         if np.dot(dir, owp_normals[k]) < 0.0:
        #             print("Inflow edge")
        #         else:
        #             print("Outflow edge")

# Finalize Gmsh
gmsh.finalize()

32
